In [1]:
# import sys
# sys.path.append('/mnt/disk1/yunseob/Pytorch/1_CapsuleEndo/')
from CE_utils import sm160_dataset, GenerateLabel
from CE_training import CNN_Training

import numpy as np
from itertools import product

In [2]:
def sm160_load(tv_rate = 0.9):
    neg_train_aug_path, neg_train_x, neg_valid_x = sm160.Get_train_valid_set('n', 'neg', train_rate = tv_rate)
    redspot_train_aug_path, redspot_train_x, redspot_valid_x = sm160.Get_train_valid_set('h', 'redspot', train_rate = tv_rate)
    angio_train_aug_path, angio_train_x, angio_valid_x = sm160.Get_train_valid_set('h', 'angio', train_rate = tv_rate)
    active_train_aug_path, active_train_x, active_valid_x = sm160.Get_train_valid_set('h', 'active', train_rate = tv_rate)
    ero_train_aug_path, ero_train_x, ero_valid_x = sm160.Get_train_valid_set('d', 'ero', train_rate = tv_rate)
    ulcer_train_aug_path, ulcer_train_x, ulcer_valid_x = sm160.Get_train_valid_set('d', 'ulcer', train_rate = tv_rate)
    str_train_aug_path, str_train_x, str_valid_x = sm160.Get_train_valid_set('d', 'str', train_rate = tv_rate)

    # 실제 모델이 학습할 augmented training data의 경로를 클래스 기준에 따라 list로 정리  
    h_train_aug_path = np.hstack([redspot_train_aug_path, angio_train_aug_path, active_train_aug_path])
    d_train_aug_path = np.hstack([ero_train_aug_path, ulcer_train_aug_path, str_train_aug_path])
    a_train_aug_path = np.hstack([h_train_aug_path, d_train_aug_path])
    train_paths = [neg_train_aug_path, a_train_aug_path]

    # epcch마다 모델을 평가할 (not augmented) training set과 validation set 클래스 기준에 따라 정리
    h_train_X = np.vstack([redspot_train_x, angio_train_x, active_train_x])
    d_train_X = np.vstack([ero_train_x, ulcer_train_x, str_train_x])
    a_train_X = np.vstack([h_train_X, d_train_X])

    h_valid_X = np.vstack([redspot_valid_x, angio_valid_x, active_valid_x])
    d_valid_X = np.vstack([ero_valid_x, ulcer_valid_x, str_valid_x])
    a_valid_X = np.vstack([h_valid_X, d_valid_X])

    train_X = [neg_train_x, a_train_X]
    valid_X = [neg_valid_x, a_valid_X]
    
    return train_paths, train_X, valid_X

In [5]:
phase_a = [[1, 1, 1]]
phase_a_label = ['---_sv']

sm160 = sm160_dataset(phase = 'train', data = 'sm_x160_v2', pre_a = phase_a[0], pre_b = False, ext_name = False)
train_paths, train_X, valid_X = sm160_load()

negative - negative | augmented training data: 179808 |(For validation) train x: (5619, 3, 512, 512) valid x: (625, 3, 512, 512)
hemorrhagic - red_spot | augmented training data: 16000 |(For validation) train x: (500, 3, 512, 512) valid x: (56, 3, 512, 512)
hemorrhagic - angioectasia | augmented training data: 512 |(For validation) train x: (16, 3, 512, 512) valid x: (2, 3, 512, 512)
hemorrhagic - active_bleeding | augmented training data: 12320 |(For validation) train x: (385, 3, 512, 512) valid x: (43, 3, 512, 512)
depressed - erosion | augmented training data: 21216 |(For validation) train x: (663, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - ulcer | augmented training data: 21056 |(For validation) train x: (658, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - stricture | augmented training data: 2944 |(For validation) train x: (92, 3, 512, 512) valid x: (11, 3, 512, 512)


In [3]:
phase_a = [[0, 0, 0]]
phase_a_label = ['---_sv']

sm160 = sm160_dataset(phase = 'train', data = 'sm_x160_v2', pre_a = phase_a[0], pre_b = True, ext_name = False)
train_paths, train_X, valid_X = sm160_load()

negative - negative | augmented training data: 28095 |(For validation) train x: (5619, 3, 512, 512) valid x: (625, 3, 512, 512)
hemorrhagic - red_spot | augmented training data: 2500 |(For validation) train x: (500, 3, 512, 512) valid x: (56, 3, 512, 512)
hemorrhagic - angioectasia | augmented training data: 80 |(For validation) train x: (16, 3, 512, 512) valid x: (2, 3, 512, 512)
hemorrhagic - active_bleeding | augmented training data: 1925 |(For validation) train x: (385, 3, 512, 512) valid x: (43, 3, 512, 512)
depressed - erosion | augmented training data: 3315 |(For validation) train x: (663, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - ulcer | augmented training data: 3290 |(For validation) train x: (658, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - stricture | augmented training data: 460 |(For validation) train x: (92, 3, 512, 512) valid x: (11, 3, 512, 512)


In [7]:
train_paths[0][:10]

array(['/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/SH___04-14-04___1030495__c_-_-_-.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/SH___04-14-04___1030495__c_-_-_-_s_-30_v_-30.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/SH___04-14-04___1030495__c_-_-_-_s_-30_v_30.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/SH___04-14-04___1030495__c_-_-_-_s_30_v_-30.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/SH___04-14-04___1030495__c_-_-_-_s_30_v_30.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/train/negative/negative/PM___02-32-03___2027881__c_-_-_-.jpg',
       '/mnt/disk2/data/private_data/SMhospital/capsule/1 preprocessed/sm_x160_v2/trai

In [4]:
phase_a = [[0, 0, 0]]
phase_a_label = ['---_sv']

sm160 = sm160_dataset(phase = 'train', data = 'sm_x160_v2', pre_a = phase_a[0], pre_b = True, ext_name = False)
train_paths, train_X, valid_X = sm160_load()

negative - negative | augmented training data: 899040 |(For validation) train x: (5619, 3, 512, 512) valid x: (625, 3, 512, 512)
hemorrhagic - red_spot | augmented training data: 80000 |(For validation) train x: (500, 3, 512, 512) valid x: (56, 3, 512, 512)
hemorrhagic - angioectasia | augmented training data: 2560 |(For validation) train x: (16, 3, 512, 512) valid x: (2, 3, 512, 512)
hemorrhagic - active_bleeding | augmented training data: 61600 |(For validation) train x: (385, 3, 512, 512) valid x: (43, 3, 512, 512)
depressed - erosion | augmented training data: 106080 |(For validation) train x: (663, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - ulcer | augmented training data: 105280 |(For validation) train x: (658, 3, 512, 512) valid x: (74, 3, 512, 512)
depressed - stricture | augmented training data: 14720 |(For validation) train x: (92, 3, 512, 512) valid x: (11, 3, 512, 512)
